# Multi-index
## Wat is een multi-index?
Een multi-index is een index die uit meer dan 1 veld bestaat

In [1]:
import pandas as pd
data = {
    'level1':[1, 1, 1, 2, 2, 2],
    'level2':[1, 2, 3, 1, 2, 3],
    'data': [30, 40, 20, 10, 50, 10]
}
df = pd.DataFrame(data)
df_multi = df.set_index(['level1', 'level2'])
df_multi

data
level1 level2      
1      1         30
       2         40
       3         20
2      1         10
       2         50
       3         10

## We kunnen de index-niveaus apart aanspreken
Net zoals vroeger gebruiken we *.loc[]* om een bepaalde indexwaarde te gebruiken. Door meerdere *.loc*'s achter elkaar te zetten kunnen we telkens een niveau verder gaan. 

In [3]:
display('alle records met level1= 2')
display(df_multi.loc[2])
display('data met level1 = 2 en level2=3')
display(df_multi.loc[(2, 3)])

'alle records met level1= 2'

,data
level2,
1,10
2,50
3,10


'data met level1 = 2 en level2=3'

data    10
Name: (2, 3), dtype: int64

## Een multi-index maken en gebruiken voor een Series
We hebben net gezien dat we een multi-index kunnen maken door twee lists te combineren. Het probleem is dat we de waarden van de hogere niveaus meerdere keren moeten herhalen: 'level1':[1, 1, 1, 2, 2, 2]

Er is een functie in pd.MultiIndex die ons kan helpen:

In [4]:
import numpy as np
import pandas as pd
jaar = [2025, 2026]
kwartaal = [1, 2, 3, 4]
maand = [1, 2, 3]
index = pd.MultiIndex.from_product([jaar, kwartaal, maand], names=['jaar', 'kwartaal', 'maand'])
index.__len__()
rng = np.random.default_rng(42)
data = rng.integers(1000, 10_000, size=len(index))
ser = pd.Series(data, index= index)
ser

jaar  kwartaal  maand
2025  1         1        1803
                2        7965
                3        6891
      2         1        4949
                2        4897
                3        8727
      3         1        1773
                2        7276
                3        2813
      4         1        1847
                2        5738
                3        9780
2026  1         1        7621
                2        7850
                3        7457
      2         1        8074
                2        5619
                3        2153
      3         1        8557
                2        5053
                3        5503
      4         1        4337
                2        2642
                3        9340
dtype: int64

## Tussendoor: de namen van de maanden
In het vorige voorbeeld wou ik de omzet per jaar, per kwartaal en per maand tonen. Hoe ik de namen van de maanden gebruiken als derde niveau?
- ik begin met een dataframe te maken met de namen van de maanden voor de 2 jaren
- Vervolgens combineer ik de kolommen van het dataframe en de series
- Met set_index(,append=True) kan ik een kolom toevoegen aan de index. 
- Met index.droplevel(-2) (of droplevel(2)) verwijder ik de index 'maand'. Dat geeft een nieuwe index terug
- Die index gebruik ik voor het dataframe

In [5]:
maanden = ['jan', 'feb', 'maa', 'apr', 'mei', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dec'] * 2 # 2 jaar
df = pd.DataFrame(maanden, index = ser.index, columns=["maandnaam"])
df = pd.concat([df, ser], axis=1)
df = df.rename(columns={0:'omzet'})
df = df.set_index('maandnaam', append=True)
df = df.set_index(df.index.droplevel(-2))
df

omzet
jaar kwartaal maandnaam       
2025 1        jan         1803
              feb         7965
              maa         6891
     2        apr         4949
              mei         4897
              jun         8727
     3        jul         1773
              aug         7276
              sep         2813
     4        okt         1847
              nov         5738
              dec         9780
2026 1        jan         7621
              feb         7850
              maa         7457
     2        apr         8074
              mei         5619
              jun         2153
     3        jul         8557
              aug         5053
              sep         5503
     4        okt         4337
              nov         2642
              dec         9340

## MultiIndex gebruiken voor berekeningen
Nu kan ik bijvoorbeeld de omzet per jaar of per kwartaal berekenen. Hiervoor gebruiken we *groupby* gecombineerd met *level*

In [6]:
display(df.groupby(level=0).sum())
display(df.groupby(level=[0, 1]).sum())
display(df.groupby(level=[0, 1, 2], sort=False).sum()) #dit heeft niet veel zin, natuurlijk (sort=False om niet te sorteren)

,omzet
jaar,
2025,64459
2026,74206


omzet
jaar kwartaal       
2025 1         16659
     2         18573
     3         11862
     4         17365
2026 1         22928
     2         15846
     3         19113
     4         16319

omzet
jaar kwartaal maandnaam       
2025 1        jan         1803
              feb         7965
              maa         6891
     2        apr         4949
              mei         4897
              jun         8727
     3        jul         1773
              aug         7276
              sep         2813
     4        okt         1847
              nov         5738
              dec         9780
2026 1        jan         7621
              feb         7850
              maa         7457
     2        apr         8074
              mei         5619
              jun         2153
     3        jul         8557
              aug         5053
              sep         5503
     4        okt         4337
              nov         2642
              dec         9340

## Unstack()
Met unstack() kunnen we een indexlevel overbrengen naar de kolommen. In dit geval doen we dat met 2 levels omdat de maanden natuurlijk niet herhaald worden per kwartaal.

In [7]:
df = df.unstack(level=[1, 2], sort=False)
df

omzet                                                              \
kwartaal      1                 2                 3                 4         
maandnaam   jan   feb   maa   apr   mei   jun   jul   aug   sep   okt   nov   
jaar                                                                          
2025       1803  7965  6891  4949  4897  8727  1773  7276  2813  1847  5738   
2026       7621  7850  7457  8074  5619  2153  8557  5053  5503  4337  2642   

                 
kwartaal         
maandnaam   dec  
jaar             
2025       9780  
2026       9340

## Een multicolumn
Aangezien een column hetzelfde is als een index, kunnen we hier ook een tuple gebruiken om de verschillende levels te selecteren:

In [22]:
print("df.loc[2025, ('omzet', 1)]:")
print(df.loc[2025, ('omzet', 1)])
print("\ndf.loc[2025, ('omzet', 1, 'feb')]:")
print(df.loc[2025, ('omzet', 1, 'feb')], sep='\n')

df.loc[2025, ('omzet', 1)]:
maandnaam
jan    1803
feb    7965
maa    6891
Name: 2025, dtype: int64

df.loc[2025, ('omzet', 1, 'feb')]:
7965


## Stack()
De functie stack() brengt een kolom over naar de index. Om geen onnodige NA-waardente krijgen, geven we *future_stack=True* mee. Dit zorgt ervoor dat de nieuwe versie van *stack* gebruikt wordt.

Bij de laatste .stack() worden er NA-waarden gegenereerd. Dat zou eigenlijk niet mogen dankzij future_stack=True. Maar dat kunnen we gemakkelijk oplossen.

In [ ]:
df = df.stack(future_stack=True, level=1)
df = df.stack(future_stack=True)
df = df.dropna()

omzet
jaar kwartaal maandnaam        
2025 1        jan        1803.0
              feb        7965.0
              maa        6891.0
              apr           NaN
              mei           NaN
...                         ...
2026 4        aug           NaN
              sep           NaN
              okt        4337.0
              nov        2642.0
              dec        9340.0

[96 rows x 1 columns]

## Datums en multiindex
In de praktijk vertrekken we dikwijls van een dataframe met datums en willen we een multiindex maken op basis van jaar, kwartaal, maand, ... We maken een dataframe met de omzetten voor 2025 en 2026

In [75]:
import numpy as np
import pandas as pd
daterange = pd.date_range(pd.to_datetime('2025-01-01'), pd.to_datetime('2026-12-31'))
rng = np.random.default_rng(42)
omzet = rng.integers(1000, 10000, size=len(daterange))
df = pd.DataFrame({'datum': daterange, 'omzet':omzet})
df.head()

,datum,omzet
0,2025-01-01,1803
1,2025-01-02,7965
2,2025-01-03,6891
3,2025-01-04,4949
4,2025-01-05,4897


## jaren, kwartalen en maanden
We maken aparte kolommen aan voor de jaren, de kwartalen en de maanden

In [76]:
df['jaar'] = df.datum.dt.year
df['kwartaal'] = df.datum.dt.quarter
df['maand'] = df.datum.dt.month
df.head()

,datum,omzet,jaar,kwartaal,maand
0,2025-01-01,1803,2025,1,1
1,2025-01-02,7965,2025,1,1
2,2025-01-03,6891,2025,1,1
3,2025-01-04,4949,2025,1,1
4,2025-01-05,4897,2025,1,1


In [ ]:
df = df.set_index(['jaar', 'kwartaal', 'maand'])
df

## .loc en multiindex
Let op wanneer je indexwaarden van verschillende niveaus wil gebruiken met .loc. De volgende code:
```
df.loc[[2025, 2]]
```
gebruikt advanced indexing. Je moet de indexwaarden als een tuple invullen. In dit geval tonen we het tweede kwartaal van 2025

In [85]:
df.loc[(2025, 2)]

,datum,omzet
maand,,
4,2025-04-01,4916
4,2025-04-02,8242
4,2025-04-03,8575
4,2025-04-04,4487
4,2025-04-05,9082
...,...,...
6,2025-06-26,7453
6,2025-06-27,2451
6,2025-06-28,9105


## Om het moeilijker te maken
Ik wil de eerste maand van elk kwartaal van 2026 zien: het jaar 2026, alle kwartalen(slice(None)) en elke eerste maand: voor het eerste kwartaal is dat 1, voor het tweede kwartaal 4, voor het derde kwartaal 7 en voor het vierde kwartaal 10

In [92]:
df.loc[(2026, slice(None), [1, 4, 7, 10])]

datum  omzet
jaar kwartaal maand                  
2026 1        1     2026-01-01   5669
              1     2026-01-02   5225
              1     2026-01-03   3843
              1     2026-01-04   1199
              1     2026-01-05   7948
...                        ...    ...
     4        10    2026-10-27   6377
              10    2026-10-28   7253
              10    2026-10-29   5529
              10    2026-10-30   8746
              10    2026-10-31   9270

[123 rows x 2 columns]